In [1]:
# example of fitting an auxiliary classifier gan (ac-gan) on fashion mnsit
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
import os
import gzip
import numpy as np
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
import matplotlib.pyplot as plt

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    # downsample to 14x14
    fe = Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # normal
    fe = Conv2D(64, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # downsample to 7x7
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # normal
    fe = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # real/fake output
    out1 = Dense(1, activation='sigmoid')(fe)
    # class label output
    out2 = Dense(n_classes, activation='softmax')(fe)
    # define model
    model = Model(in_image, [out1, out2])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy','sparse_categorical_crossentropy'], optimizer=opt)
    return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=10):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # linear multiplication
    n_nodes = 7 * 7
    li = Dense(n_nodes, kernel_initializer=init)(li)
    # reshape to additional channel
    li = Reshape((7, 7, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 384 * 7 * 7
    gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
    gen = Activation('relu')(gen)
    gen = Reshape((7, 7, 384))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 14x14
    gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
    gen = BatchNormalization()(gen)
    gen = Activation('relu')(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    out_layer = Activation('tanh')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect the outputs of the generator to the inputs of the discriminator
    gan_output = d_model(g_model.output)
    # define gan model as taking noise and label and outputting real/fake and label outputs
    model = Model(g_model.input, gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

Using TensorFlow backend.


In [2]:
def load_real_samples(path, kind='train'):
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    images = images.astype('float32')
    images = images.reshape(60000, 28, 28, 1)
    # scale from [0,255] to [-1,1]
    images = (images - 127.5) / 127.5
    return [images, labels]

In [10]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = np.ones((n_samples, 1))
    return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = np.zeros((n_samples, 1))
    return [images, labels_input], y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    # prepare fake examples
    [X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(100):
        # define subplot
        plt.subplot(10, 10, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename1 = 'generated_plot_%04d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
    # save the generator model
    filename2 = 'model_%04d.h5' % (step+1)
    g_model.save(filename2)

In [14]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=5, n_batch=64):
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_steps):
        # get randomly selected ✬real✬ samples
        [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        _,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
        # generate ✬fake✬ examples
        [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # update discriminator model weights
        _,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
        # prepare points in latent space as input for the generator
        [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        # update the generator via the discriminator✬s error
        _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
        # summarize loss on this batch
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2, d_f,d_f2,g_1,g_2))
        # evaluate the model performance every ✬epoch✬
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, latent_dim)

In [15]:
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples('fash')

In [16]:
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

C:\Users\VG Rig\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\VG Rig\anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


>1, dr[0.952,2.838], df[1.853,3.218], g[0.638,3.143]
>2, dr[0.488,3.154], df[1.360,3.012], g[0.914,2.933]
>3, dr[0.611,3.082], df[0.974,3.581], g[0.848,3.078]
>4, dr[1.130,2.851], df[0.865,2.884], g[1.038,3.076]


C:\Users\VG Rig\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>5, dr[0.765,3.147], df[0.634,2.881], g[1.135,2.984]
>6, dr[0.888,2.795], df[0.997,3.500], g[1.137,2.938]
>7, dr[0.680,2.796], df[0.877,3.368], g[1.035,3.064]
>8, dr[0.650,2.801], df[0.594,3.211], g[1.400,3.288]
>9, dr[0.657,3.167], df[0.637,3.074], g[1.208,3.121]
>10, dr[0.907,2.447], df[0.541,3.122], g[1.142,3.380]
>11, dr[0.476,2.518], df[0.459,2.981], g[1.160,3.234]
>12, dr[0.466,2.441], df[0.532,3.206], g[1.296,3.176]
>13, dr[0.508,2.914], df[0.530,3.127], g[1.202,3.164]
>14, dr[0.238,2.620], df[0.532,3.103], g[1.465,3.228]
>15, dr[0.420,2.596], df[0.506,2.850], g[1.400,2.941]
>16, dr[0.619,2.325], df[0.438,2.757], g[1.471,2.948]
>17, dr[0.321,2.646], df[0.662,2.828], g[1.500,3.254]
>18, dr[0.386,2.486], df[0.464,2.964], g[1.770,3.044]
>19, dr[0.411,2.492], df[0.370,3.400], g[1.689,3.145]
>20, dr[0.587,2.315], df[0.424,3.169], g[1.388,3.236]
>21, dr[0.324,1.762], df[0.441,2.869], g[1.330,2.852]
>22, dr[0.383,2.135], df[0.518,3.323], g[1.024,3.008]
>23, dr[0.323,1.606], df[0.294,3.

>156, dr[0.071,1.150], df[0.115,0.550], g[0.094,0.533]
>157, dr[0.107,1.076], df[0.204,0.639], g[0.120,0.626]
>158, dr[0.182,1.376], df[0.108,0.553], g[0.090,0.536]
>159, dr[0.140,0.688], df[0.032,0.679], g[0.116,0.483]
>160, dr[0.031,0.830], df[0.078,0.419], g[0.056,0.356]
>161, dr[0.133,0.625], df[0.150,0.383], g[0.039,0.594]
>162, dr[0.072,1.524], df[0.087,0.256], g[0.028,0.627]
>163, dr[0.204,0.917], df[0.011,0.674], g[0.043,0.547]
>164, dr[0.133,0.620], df[0.135,0.316], g[0.070,0.388]
>165, dr[0.077,1.130], df[0.044,0.379], g[0.050,0.304]
>166, dr[0.038,0.927], df[0.239,0.333], g[0.137,0.377]
>167, dr[0.162,0.612], df[0.103,0.362], g[0.076,0.346]
>168, dr[0.149,1.042], df[0.044,0.375], g[0.070,0.313]
>169, dr[0.078,0.700], df[0.088,0.402], g[0.059,0.326]
>170, dr[0.106,1.319], df[0.134,0.422], g[0.074,0.228]
>171, dr[0.073,1.183], df[0.023,0.453], g[0.028,0.292]
>172, dr[0.152,1.040], df[0.098,0.345], g[0.048,0.276]
>173, dr[0.063,1.300], df[0.134,0.595], g[0.107,0.264]
>174, dr[0

...

>4022, dr[0.647,0.662], df[0.518,0.079], g[1.473,0.206]
>4023, dr[0.403,0.271], df[0.466,0.030], g[1.291,0.100]
>4024, dr[0.809,0.202], df[0.725,0.021], g[1.714,0.127]
>4025, dr[0.412,0.184], df[0.690,0.118], g[1.807,0.062]
>4026, dr[0.443,0.397], df[0.527,0.086], g[1.822,0.062]
>4027, dr[0.710,0.824], df[0.380,0.027], g[1.690,0.091]
>4028, dr[0.337,0.600], df[0.524,0.024], g[1.571,0.081]
>4029, dr[0.537,0.314], df[0.622,0.038], g[1.794,0.063]
>4030, dr[0.451,0.358], df[0.614,0.227], g[1.755,0.106]
>4031, dr[0.667,0.320], df[0.640,0.068], g[1.977,0.141]
>4032, dr[0.759,0.683], df[0.494,0.028], g[1.692,0.072]
>4033, dr[0.407,0.265], df[0.463,0.073], g[1.523,0.093]
>4034, dr[0.435,0.764], df[0.498,0.018], g[1.610,0.039]
>4035, dr[0.505,0.662], df[0.395,0.030], g[1.804,0.026]
>4036, dr[0.357,0.477], df[0.680,0.093], g[2.123,0.102]
>4037, dr[0.506,0.676], df[0.544,0.122], g[2.276,0.083]
>4038, dr[1.098,0.715], df[0.511,0.066], g[1.926,0.039]
>4039, dr[0.631,0.549], df[0.376,0.010], g[1.405

>4169, dr[0.944,0.549], df[0.514,0.078], g[1.565,0.134]
>4170, dr[0.612,0.363], df[0.692,0.018], g[1.399,0.075]
>4171, dr[0.407,0.680], df[0.790,0.055], g[1.588,0.111]
>4172, dr[0.860,0.900], df[0.334,0.007], g[1.531,0.045]
>4173, dr[0.633,0.521], df[0.688,0.117], g[1.487,0.068]
>4174, dr[0.357,0.347], df[0.621,0.009], g[1.355,0.128]
>4175, dr[0.388,0.416], df[0.388,0.056], g[1.565,0.018]
>4176, dr[0.591,0.604], df[0.469,0.026], g[1.732,0.056]
>4177, dr[0.600,0.440], df[0.648,0.052], g[1.516,0.042]
>4178, dr[0.653,0.679], df[0.839,0.011], g[1.372,0.044]
>4179, dr[0.537,0.620], df[0.417,0.159], g[1.732,0.096]
>4180, dr[0.697,0.497], df[0.628,0.176], g[1.080,0.077]
>4181, dr[0.584,1.198], df[0.726,0.022], g[1.410,0.242]
>4182, dr[0.477,0.828], df[0.323,0.037], g[1.508,0.031]
>4183, dr[0.358,0.248], df[0.401,0.038], g[2.073,0.018]
>4184, dr[0.379,0.473], df[0.299,0.007], g[1.567,0.049]
>4185, dr[0.370,0.264], df[0.745,0.054], g[1.539,0.120]
>4186, dr[0.679,0.933], df[0.513,0.222], g[1.689

>4317, dr[0.633,0.499], df[0.669,0.018], g[1.598,0.023]
>4318, dr[0.404,0.434], df[0.966,0.104], g[1.931,0.159]
>4319, dr[0.779,0.571], df[0.335,0.130], g[1.531,0.042]
>4320, dr[0.717,0.303], df[0.570,0.087], g[1.583,0.186]
>4321, dr[0.497,0.594], df[0.566,0.158], g[1.522,0.067]
>4322, dr[0.632,0.442], df[0.410,0.016], g[1.692,0.044]
>4323, dr[0.706,0.547], df[1.002,0.164], g[1.459,0.072]
>4324, dr[0.278,0.200], df[0.350,0.252], g[1.630,0.010]
>4325, dr[0.690,0.773], df[0.347,0.144], g[1.796,0.091]
>4326, dr[0.371,0.196], df[0.477,0.262], g[1.376,0.099]
>4327, dr[0.468,0.337], df[0.475,0.053], g[1.165,0.058]
>4328, dr[0.526,0.570], df[0.571,0.163], g[1.655,0.025]
>4329, dr[0.476,0.346], df[0.487,0.105], g[1.771,0.088]
>4330, dr[0.382,1.225], df[0.680,0.194], g[1.839,0.111]
>4331, dr[0.697,0.523], df[0.436,0.034], g[1.580,0.038]
>4332, dr[0.586,0.546], df[0.578,0.156], g[1.681,0.061]
>4333, dr[0.677,1.114], df[0.484,0.021], g[1.878,0.043]
>4334, dr[0.636,0.377], df[0.395,0.046], g[1.503

>4464, dr[0.484,0.467], df[0.504,0.063], g[1.318,0.130]
>4465, dr[0.555,0.650], df[0.741,0.137], g[1.840,0.075]
>4466, dr[0.669,0.623], df[0.431,0.202], g[1.782,0.052]
>4467, dr[0.567,0.113], df[0.511,0.119], g[2.054,0.043]
>4468, dr[0.603,0.620], df[0.529,0.087], g[1.632,0.071]
>4469, dr[0.608,0.535], df[0.756,0.064], g[1.756,0.026]
>4470, dr[0.485,0.334], df[0.428,0.017], g[1.716,0.068]
>4471, dr[0.606,0.411], df[0.528,0.228], g[1.543,0.084]
>4472, dr[0.548,0.750], df[0.609,0.087], g[1.522,0.139]
>4473, dr[0.682,0.810], df[0.589,0.049], g[1.680,0.028]
>4474, dr[0.491,0.386], df[0.638,0.068], g[1.713,0.043]
>4475, dr[0.804,0.878], df[0.603,0.025], g[1.738,0.077]
>4476, dr[0.603,0.795], df[0.589,0.259], g[1.207,0.103]
>4477, dr[0.498,0.916], df[0.516,0.020], g[1.477,0.116]
>4478, dr[0.575,0.658], df[0.846,0.107], g[1.417,0.079]
>4479, dr[0.641,0.527], df[0.683,0.087], g[1.744,0.053]
>4480, dr[0.727,0.459], df[0.508,0.085], g[1.727,0.015]
>4481, dr[0.607,0.873], df[0.839,0.247], g[2.005

>4611, dr[0.526,0.565], df[0.525,0.037], g[1.613,0.197]
>4612, dr[0.540,0.707], df[0.853,0.099], g[1.842,0.043]
>4613, dr[0.502,0.349], df[0.472,0.066], g[1.828,0.100]
>4614, dr[0.663,0.626], df[0.455,0.129], g[1.495,0.128]
>4615, dr[0.489,0.795], df[0.599,0.123], g[1.758,0.143]
>4616, dr[0.429,0.414], df[0.500,0.056], g[1.737,0.140]
>4617, dr[0.528,0.789], df[0.617,0.420], g[1.736,0.022]
>4618, dr[0.615,0.377], df[0.370,0.030], g[1.653,0.097]
>4619, dr[0.592,0.205], df[0.748,0.029], g[1.848,0.062]
>4620, dr[0.586,0.559], df[0.682,0.010], g[1.841,0.164]
>4621, dr[0.520,0.400], df[0.811,0.048], g[1.785,0.072]
>4622, dr[0.809,0.446], df[0.475,0.135], g[1.803,0.060]
>4623, dr[0.852,0.389], df[0.575,0.083], g[1.132,0.160]
>4624, dr[0.461,0.708], df[0.715,0.058], g[1.235,0.156]
>4625, dr[0.518,0.504], df[0.479,0.081], g[1.590,0.087]
>4626, dr[0.417,0.665], df[0.458,0.017], g[1.419,0.039]
>4627, dr[0.379,0.442], df[0.474,0.040], g[1.486,0.066]
>4628, dr[0.517,0.416], df[0.404,0.030], g[1.388

## Test

In [17]:
# example of loading the generator model and generating images
from math import sqrt
from numpy import asarray
from numpy.random import randn
from keras.models import load_model
import matplotlib.pyplot as plt

In [18]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_class):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = asarray([n_class for _ in range(n_samples)])
    return [z_input, labels]

# create and save a plot of generated images
def save_plot(examples, n_examples):
    # plot images
    for i in range(n_examples):
        # define subplot
        pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    pyplot.show()

# load model
model = load_model('model_4685.h5')
latent_dim = 100
n_examples = 100 # must be a square
n_class = 7 # sneaker
# generate images
latent_points, labels = generate_latent_points(latent_dim, n_examples, n_class)
# generate images
X = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, n_examples)

C:\Users\VG Rig\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


NameError: name 'pyplot' is not defined